In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import os
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import optuna

In [ ]:
df = pd.read_csv('/workspaces/python-env/work/data/Sensor2.csv', header=None, delimiter='\t')
# df = pd.read_csv('/workspaces/python-env/work/data/sample_data.txt', header=None, delimiter='\t')
df.head()

In [ ]:
# ecg_dataset = df.iloc[:,2].values
# ecg_dataset = np.array(ecg_dataset)
ecg_dataset = df.values
ecg_dataset = ecg_dataset.reshape(len(ecg_dataset), -1)

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(ecg_dataset, color='b')
plt.show()

In [ ]:
# 平均値を0にする
ecg_dataset_mean = ecg_dataset.mean()
ecg_dataset = (ecg_dataset - ecg_dataset_mean)

# データセットの最大値で割り、-1～1の範囲に収まるように正規化する
ecg_dataset_max = np.abs(ecg_dataset).max()
ecg_dataset = ecg_dataset / ecg_dataset_max

In [ ]:
# 学習データとテストデータに分割
# 0～5000をテスト用データ
test_data = ecg_dataset[:80]
# 5001～10000を学習用データ
train_data = ecg_dataset[80:120]
# train_data,test_data = train_test_split(ecg_dataset,test_size=0.2,shuffle=False)

In [ ]:
# 学習データプロット
plt.figure(figsize=(16, 9))
plt.ylim(-1, 1)
plt.plot(train_data, color='b')
plt.show()

In [ ]:
# テストデータプロット
plt.figure(figsize=(16, 9))
plt.ylim(-1, 1)
plt.plot(test_data, color='b')
plt.show()

In [ ]:

TIME_STEPS = 12

def create_dataset(array, time_steps=TIME_STEPS):
    dataset_list = []

    for i in range(len(array) - time_steps + 1):
        dataset_list.append(array[i : (i + time_steps)])

    return np.stack(dataset_list)

# 学習データセットとテストデータセットを作成
x_train = create_dataset(train_data)
print("train_data: ",train_data.shape)
print("x_train: ",x_train.shape)

x_test = create_dataset(test_data)
print("test_data: ", test_data.shape)
print("x_test: ", x_test.shape)

In [ ]:
seed = 42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
model = tf.keras.Sequential(
    [
        # 入力層
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS, 1)),
     
        # エンコード層
        tf.keras.layers.Conv1D(
            filters=64, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Conv1D(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
       
        # デコード層
        tf.keras.layers.Conv1DTranspose(
            filters=64, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Conv1DTranspose(
            filters=64, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
    
        # 出力層
        tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
    ]
)

In [ ]:
model.compile(optimizer="adam", loss="mse")
# モデルの構造確認
model.summary()

In [ ]:
# 訓練
history = model.fit(
    x=x_train,  # 学習データ
    y=x_train,  # 教師データ(オートエンコーダの学習のため学習データをそのまま入れる)
    validation_split=0.1,  # 検証データ比率(学習データの1割を検証データとして使用する)
    epochs=50,  # エポック数
    batch_size=128,  # バッチサイズ
    callbacks=[
        # コールバック指定
        # 5回検証Lossの改善が無かったら学習を打ち切るよう設定
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
    ],
)

plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

In [ ]:
# 訓練したモデルで推論
temp_data = x_train[0]
temp_data = temp_data.reshape(-1, TIME_STEPS, 1)

temp_result = model.predict(temp_data)

plt.plot(temp_data[0])
plt.plot(temp_result[0])
plt.show()

In [ ]:
x_train_predict_result = model.predict(x_train)

train_abs = np.abs(x_train_predict_result - x_train)
train_mae = np.mean(train_abs, axis=1)
train_mae = train_mae.reshape(-1)

plt.hist(train_mae, bins=50)
plt.xlabel("Train MAE")
plt.show()

In [ ]:
threshold = np.max(train_mae)
print(threshold)

In [ ]:
x_test_predict_result = model.predict(x_test)

test_abs = np.abs(x_test_predict_result - x_test)
test_mae = np.mean(test_abs, axis=1)
test_mae = test_mae.reshape(-1)

plt.hist(test_mae, bins=50)
plt.xlabel("Test MAE")
plt.show()

In [ ]:
# 異常検知したデータの数と異常検知したデータのインデックスを確認
anomaly_result = test_mae > threshold

# 異常検知個数
print(np.sum(anomaly_result))

# 異常検知したインデックス
print(np.where(anomaly_result))

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(test_data, color="b")

for index, anomaly in enumerate(anomaly_result):
    if anomaly:
        x = np.arange(index, index + TIME_STEPS)
        y1 = [-1]*len(x)
        y2 = [1]*len(x)
        plt.fill_between(x, y1, y2, facecolor='r', alpha=.3)

In [ ]:
def create_model(time_steps, num_layer, num_filters, kernel_size, strides, dropout_rate, activation):
    model = tf.keras.Sequential()

    # 入力層
    model.add(tf.keras.layers.InputLayer(input_shape=(time_steps, 1)))

    # エンコード層
    for i in range(num_layer):
        filters = int(num_filters / (i+1))

        model.add(
            tf.keras.layers.Conv1D(
                filters=filters, kernel_size=kernel_size, padding="same", strides=strides, activation=activation
            )
        )
        if i < (num_layer - 1):
            model.add(tf.keras.layers.Dropout(rate=dropout_rate))

    # デコード層
    for i in reversed(range(num_layer)):
        filters = int(num_filters / (i+1))

        model.add(
            tf.keras.layers.Conv1DTranspose(
                filters=filters, kernel_size=kernel_size, padding="same", strides=strides, activation=activation
            )
        )
        if i != 0:
            model.add(tf.keras.layers.Dropout(rate=dropout_rate))

    # 出力層
    model.add(
        tf.keras.layers.Conv1DTranspose(
            filters=1, kernel_size=kernel_size, padding="same"
        )
    )

    return model


In [ ]:
def objective(trial):
    # レイヤー数
    num_layer = trial.suggest_int("num_layer", 1, 3)
    # 畳み込みフィルター数
    num_filters = int(trial.suggest_categorical("num_filters", [16, 32, 64]))
    # 畳み込みカーネルサイズ
    kernel_size = trial.suggest_int("kernel_size", 1, 5, 2)
    # 畳み込みストライドサイズ
    strides = trial.suggest_int("strides", 2, 4, 2)
    # ドロップアウト割合
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
    # 活性化関数
    activation = trial.suggest_categorical("activation", ["relu", "sigmoid", "tanh"])
    
    # 最適化アルゴリズム
    optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam"])

    # モデル構築・コンパイル
    model = create_model(TIME_STEPS, num_layer, num_filters, kernel_size, strides, dropout_rate, activation)
    model.compile(
        optimizer=optimizer,
        loss="mse"
    )

    model.summary()
    
    # 訓練開始
    history = model.fit(
        x_train,
        x_train,
        epochs=50,
        batch_size=128,
        validation_split=0.1,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
        ],
    )
    
    return history.history["val_loss"][-1]


study = optuna.create_study()
study.optimize(objective, n_trials=50)

print(study.best_params)

model2 = create_model(
    TIME_STEPS, 
    study.best_params['num_layer'], 
    study.best_params['num_filters'], 
    study.best_params['kernel_size'],
    study.best_params['strides'], 
    study.best_params['dropout_rate'],
    study.best_params['activation'],
)

model2.compile(optimizer=study.best_params['optimizer'], loss="mse")

model2.summary()

history = model2.fit(
    x_train,
    x_train,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
    ],
)

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

In [ ]:
x_train_predict_result2 = model2.predict(x_train)

train_abs2 = np.abs(x_train_predict_result2 - x_train)
train_mae2 = np.mean(train_abs2, axis=1)
train_mae2 = train_mae2.reshape(-1)

plt.hist(train_mae2, bins=50)
plt.xlabel("Train MAE")
plt.show()

In [ ]:
threshold2 = np.max(train_mae2)
print(threshold2)

In [ ]:
x_test_predict_result2 = model2.predict(x_test)

test_abs2 = np.abs(x_test_predict_result2 - x_test)
test_mae2 = np.mean(test_abs2, axis=1)
test_mae2 = test_mae2.reshape(-1)

plt.hist(test_mae2, bins=50)
plt.xlabel("Test MAE")
plt.show()

In [ ]:
anomaly_result2 = test_mae2 > threshold2
print(np.sum(anomaly_result2))
print(np.where(anomaly_result2))

In [ ]:
plt.figure(figsize=(16, 9))

plt.plot(test_data, color="b")

for index, anomaly in enumerate(anomaly_result2):
    if anomaly:
        x = np.arange(index, index + TIME_STEPS)
        y1 = [-1]*len(x)
        y2 = [1]*len(x)
        plt.fill_between(x, y1, y2, facecolor='r', alpha=.3)